In [ ]:
"""
Chapter 05: Scrapy
Date: 25/05/2024
"""

# initializing a new spider

To create a new spider in the current directory, run the following from the commandline:

- scrapy startproject wikiSpider

## writing a simple scraper

In [2]:
'''To create a crawler, you will add a new file inside the spiders directory 
at wikiSpider/wikiSpider/spiders/article.py. In your newly created article.py 
file, write the following:'''

import scrapy


class ArticleSpider(scrapy.Spider):
    name = 'article'

    def start_request(self):
        urls = [
            # tại đây sẽ sử dụng wikipedia để ví dụ
            'http://en.wikipedia.org/wiki/Python_''%28programming_language%29','https://en.wikipedia.org/wiki/Functional_programming','https://en.wikipedia.org/wiki/Monty_Python'
        ]
        return [scrapy.Request(url=url, callback=self.parse) for url in urls]
    
    def parse(self, response):
        url = response.url
        title = response.css('h1::text').extract_first()
        print(f'URL is: {url}')
        print(f'Title is: {title}')

# run this:
# scrapy runspider article.py

## spidering with rules

khung Scrapy không thể dễ dàng chạy từ bên trong sổ ghi chép Jupyter, khiến cho tiến trình tuyến tính của mã khó nắm bắt được

In [6]:
from scrapy.linkextractors import LinkExtractor
from scrapy.spiders import CrawlSpider, Rule


class ArticleSpider(CrawlSpider):
    name = 'article'
    allowed_domains = ['wikipedia.org']
    start_urls = ['https://en.wikipedia.org/wiki/Benevolent_dictator_for_life']
    rules = [Rule(LinkExtractor(allow=r'.*'), callback='parse_items', follow=True)]

    def parse_items(self, response):
        url = response.url
        title = response.css('h1::text').extract_first()
        text = response.xpath('//div[@id="mw-content-text"]//text()').extract_first()
        lastUpdated = lastUpdated.replace(
            'This page was last edited on ', ''
        )
        print('URL is: {}'.format(url))
        print('title is: {} '.format(title))
        print('text is: {}'.format(text))
        print('Last updated: {}'.format(lastUpdated))

'''
một Rule có thể được truyền vào 6 tham số:
    - link_extractor: đối số bắt buộc duy nhất đối tượng LinkExtractor
    - callback: hàm nên được sử dụng để phân tích nội dung trên page
    - cb_kwargs: một từ điển các đối số được truyền đến hàm callback với {arg_name1: arg_value1, arg_name2: arg_value2}
    và nó có thể hữu ích để sử dụng lại các chức năng phân tích cú pháp giống nhau cho các tác vụ hơi khác nhau
    - follow: cho biết liệu có muốn đưa liên kết được tìm thấy tại trang đó vào trình
    thu thập thông tin trong tương lai hay không. nếu không có callback thì mặc định là True, ngược lại là False

LinkExtractor là một lớp đơn giản được thiết kế chỉ để nhận dạng và trả về các liên kết 
trong một trang nội dung HTML dựa trên các quy tắc được cung cấp cho nó.   
Nó có một số đối số có thể được sử dụng để chấp nhận hoặc từ chối một liên 
kết dựa trên các bộ chọn, thẻ CSS và XPath

Lớp LinkExtractor có thể thậm chí còn được mở rộng và có thể tạo các đối số tùy chỉnh
chúng có các đối số thường dùng là:
    - allow: Allow all links that match the provided regular expression.
    - deny: Deny all links that match the provided regular expression

'''

In [ ]:
'''Bằng cách sử dụng hai lớp Rule và LinkExtractor riêng biệt với một chức 
năng phân tích cú pháp duy nhất, bạn có thể tạo một trình thu thập thông 
tin thu thập dữ liệu Wikipedia, xác định tất cả các trang bài viết và các 
trang không phải bài viết đang gắn cờ (articlesMoreRules.py)'''

from scrapy.linkextractors import LinkExtractor
from scrapy.spiders import CrawlSpider, Rule


class ArticleSpider(CrawlSpider):
    name='Chapter03_WritingWebCrawler.ipynb'
    allowed_domain = ['wikipedia.org']
    start_urls = ['https://en.wikipedia.org/wiki/Benevolent_dictator_for_life']
    rules=[
        Rule(LinkExtractor(allow='^(/wiki/)((?!:).)*$'), callback='parse_items', follow=True, cb_kwargs={'is_article':True}),
        Rule(LinkExtractor(allow='.*'),callback='parse_items',cb_kwargs={'is_article':False})
    ]

    def parse_items(self, response, is_article):
        print(response.url)
        title = response.css('h1::text').extract_first()
        if is_article:
            url=response.url
            text=response.xpath('//div[@id="mw-content-text"]//text()').extract()
            lastUpdated=response.css('li#footer-info-lastmod''::text').extract_first()
            lastUpdated=lastUpdated.replace('This page was ''last edited on ','')
            print('Title is: {} '.format(title))
            print('title is: {} '.format(title))
            print('text is: {}'.format(text))
        else:
            print('This is not an article: {}'.format(title))

# creating items

Scrapy  also  provides  useful  tools  to  keep  your  collected  items  organizedand stored in custom objects with well-defined fields.To help organize all the information you’re collecting, you need to create an Articleobject. Define a new item called Article inside the items.py file

# outputing items

$ scrapy runspider articleItems.py -o articles.csv -t csv

$ scrapy runspider articleItems.py -o articles.json -t json

$ scrapy runspider articleItems.py -o articles.xml -t xml

# the item pipeline


# logging with scrapy

Thông tin gỡ lỗi do Scrapy tạo ra có thể hữu ích, nhưng, như bạn có thể đã nhận thấy, thông tin này thường quá dài dòng. Bạn có thể dễ dàng điều chỉnh mức độ ghi nhật ký bằng cách thêm aline vào tệp settings.py trong dự án Scrapy của mình

LOG_LEVEL='ERROR'

Scrapy sử dụng hệ thống phân cấp tiêu chuẩn của các cấp độ ghi nhật ký, như sau:

    CRITICAL
    ERROR
    WARNING
    DEBUG
    INFO